<u><b><p style="color:Red; text-align:center; background-color: yellow; font-size:300%">House Prices Estimation</u></b></p>

<b>Loading The data </b>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_seq_items = 2000
pd.options.display.max_rows = 3000
pd.options.display.max_columns = 500

In [ ]:
df_train = pd.read_csv(r"C:\Kaggles\House Prices Predication\DataSet\train.csv")
df_test = pd.read_csv(r"C:\Kaggles\House Prices Predication\DataSet\test.csv")

print(df_train.shape, df_test.shape)

In [ ]:
df_train.head()

<b>Data Cleaning and Feature Engineering</b>

In [ ]:
df_train.drop('Id', axis=1, inplace=True)
Id = df_test["Id"]
df_test.drop("Id", axis=1, inplace=True)

df_test["SalePrice"] = np.nan

In [ ]:
print(df_train['SalePrice'].describe())
plt.figure(figsize=(9, 8))
sns.distplot(df_train['SalePrice'], color='g', bins=100, hist_kws={'alpha': 0.4});

In [ ]:
combined_df = pd.concat([df_train, df_test])
combined_df.shape

In [ ]:
combined_df = pd.get_dummies(combined_df)

from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for column in combined_df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    combined_df[column] = le.fit_transform(combined_df[column])
    label_encoders[column] = le

combined_df.shape

In [ ]:
df_train = combined_df[combined_df["SalePrice"].notna()]
df_test = combined_df[combined_df["SalePrice"].isna()]
print(df_train.shape, df_test.shape)

In [ ]:
df_test.drop("SalePrice", axis=1, inplace=True)

In [ ]:
print(df_train.shape, df_test.shape)

In [ ]:
df_train = df_train.fillna(df_train.median()) 
df_test = df_test.fillna(df_test.median()) 

In [ ]:
X = df_train.drop("SalePrice", axis=1)
Y = df_train.SalePrice

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.12, random_state=42)


In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ["squared_error", "friedman_mse"],
                'splitter': ['best','random']
            }
        }, 
        'RandomForestRegressor' : {
            'model' : RandomForestRegressor(),
            'params' : {
                'n_estimators': [50, 100, 200],
                'max_depth': [None, 5, 10, 20],
                'min_samples_split': [2, 5, 10],
                'max_features': ['sqrt', 'log2']
            }
        },
    }
    scores = []
    cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=42)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])
Models_Score = find_best_model_using_gridsearchcv(X, Y)
Models_Score

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

cv_split = ShuffleSplit(n_splits=10, test_size=0.12, random_state=False)
KFold_score_LR = cross_val_score(RandomForestRegressor(), X, Y, cv=cv_split)
print(KFold_score_LR)
print("\nAverage Score : ", np.mean(KFold_score_LR))

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline

def prediction(model):

    pipline = Pipeline([('model',model)])
    pipline.fit(X_train,Y_train)
    y_pred = pipline.predict(X_test)
    print(f"{model}")
    print(f"r2_score is {r2_score(Y_test, y_pred)}")
    print(f"mean_absolute_error is {mean_absolute_error(Y_test, y_pred)}")
    print(f"Root mean squared error is {np.sqrt(mean_squared_error(Y_test, y_pred))}")

In [ ]:
best_model = RandomForestRegressor()
prediction(best_model)

In [ ]:
predictions = best_model.predict(df_test)

In [ ]:
submission = pd.DataFrame({
    "Id" : Id,
    "SalePrice" : predictions
})

submission.to_csv(r"C:\Kaggles\House Prices Predication\Submissions\submission_1.csv", index = False)